## Imports, function definitions

Note: DeepMoD does not work for python3.9

In [1]:
# pip install pysindy

In [2]:
# pip install DeePyMoD

In [3]:
# pip install --upgrade tensorflow

In [4]:
# pip install tensorflow-io

In [5]:
results_path = ""

In [6]:
import matplotlib.pyplot as plt

# General imports
import numpy as np
import torch
import pandas as pd

# DeePyMoD imports
from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.data.burgers import burgers_delta
from deepymod.model.constraint import LeastSquares
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.sparse_estimators import Threshold
from deepymod.training import train
from deepymod.training.sparsity_scheduler import Periodic, TrainTest, TrainTestPeriodic

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import pickle
from sklearn.metrics import mean_squared_error
def load_data(X,y):
    return torch.from_numpy(X).float(),torch.from_numpy(y).float()

def uxt_2D_to1D(u,x,t):
    '''
    Function for reshaping u,x and t
    '''
    t_reg=[]
    x_reg=[]
    for i in range(len(t)):
        for j in range(len(x)):
            t_reg.append(t[i])
            x_reg.append(x[j])
    u_reg = u.reshape((u.size, 1))

    return u_reg,x_reg,t_reg

descr = ['',
 'u_{x}',
 'u_{xx}',
 'u_{xxx}',
 'u',
 'uu_{x}',
 'uu_{xx}',
 'uu_{xxx}',
 'u^2',
 'u^2u_{x}',
 'u^2u_{xx}',
 'u^2u_{xxx}']

def return_pde(w, rhs_description=descr, ut = 'u_t'):
    pde = ut + ' = '
    first = True
    for i in range(len(w)):
        if w[i] != 0:
            if not first:
                pde = pde + ' + '
            pde = pde + "(%05f)" % (w[i].real) + rhs_description[i] 
            first = False
    return pde


cuda


In [7]:
def error_metric(true_coeff,pred_coeff):
    wrmse = mean_squared_error(true_coeff,pred_coeff,squared=False)
    return wrmse

## Testing 2.1

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(11,13):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  300
Dataset is using device:  cuda


2023-04-28 11:08:19.377755: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 11:08:19.946119: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 11750  MSE: 1.13e-04  Reg: 1.17e-05  L1: 1.42e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097939)u_{xx} + (-0.979618)uu_{x}  rmse =  0.005913908035232153
Dataset is using device:  cuda
 19750  MSE: 1.18e-04  Reg: 7.51e-06  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098588)u_{xx} + (-0.987852)uu_{x}  rmse =  0.0035302985601154556
Dataset is using device:  cuda
  6750  MSE: 1.07e-04  Reg: 6.36e-06  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097499)u_{xx} + (-0.996193)uu_{x}  rmse =  0.0013148332491027634
Dataset is using device:  cuda
 16250  MSE: 1.08e-04  Reg: 7.80e-06  L1: 1.47e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098360)u_{xx} + (-0.982453)uu_{x}  rmse =  0.005087418735924839
Dataset is using device:  cuda
 19750  MSE: 1.13e-04  Reg: 5.81e-06  L1: 1.76e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097826)u_{xx} + (-

KeyboardInterrupt: 

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(12,13):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  200
Dataset is using device:  cuda


2023-04-29 01:03:25.818150: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 01:03:26.326637: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 10500  MSE: 1.71e-04  Reg: 1.45e-05  L1: 1.44e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.096627)u_{xx} + (-0.980470)uu_{x}  rmse =  0.005721381772438387
Dataset is using device:  cuda
 19500  MSE: 1.65e-04  Reg: 9.56e-06  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099824)u_{xx} + (-0.985198)uu_{x}  rmse =  0.004273387224520526
Dataset is using device:  cuda
  7500  MSE: 1.69e-04  Reg: 5.42e-06  L1: 1.42e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097735)u_{xx} + (-0.992580)uu_{x}  rmse =  0.0022395988346520014
Dataset is using device:  cuda
 10250  MSE: 1.64e-04  Reg: 1.18e-05  L1: 1.48e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097474)u_{xx} + (-0.989764)uu_{x}  rmse =  0.0030434270756973187
Dataset is using device:  cuda
 17000  MSE: 1.66e-04  Reg: 7.11e-06  L1: 1.49e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099724)u_{xx} + (-

## Testing 2.2

In [9]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(13,15):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  100
Dataset is using device:  cuda
 18750  MSE: 3.26e-04  Reg: 8.69e-06  L1: 1.46e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098392)u_{xx} + (-0.988861)uu_{x}  rmse =  0.0032488442155119614
Dataset is using device:  cuda
 19750  MSE: 3.32e-04  Reg: 1.08e-05  L1: 2.04e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.095504)u_{xx} + (-1.202255)uu_{x} + (0.448332)u^2u_{x}  rmse =  0.141988467626679
Dataset is using device:  cuda
  4750  MSE: 3.28e-04  Reg: 1.09e-05  L1: 1.46e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099124)u_{xx} + (-0.983910)uu_{x}  rmse =  0.004651610776278788
Dataset is using device:  cuda
 19750  MSE: 3.29e-04  Reg: 4.98e-06  L1: 1.66e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.101375)u_{xx} + (-0.991952)uu_{x}  rmse =  0.0023568883544748277
Dataset is using device:  cuda
 13750  MSE: 3.25e-04  Reg: 5.53e-06  L1: 1.45e+00 Algorithm converged. Writin

## Testing 2.3

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(15,17):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  80
Dataset is using device:  cuda


2023-04-30 12:05:30.033594: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 12:05:30.608581: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  7500  MSE: 4.20e-04  Reg: 1.48e-05  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.035340)u_{x} + (0.090814)u_{xx} + (-0.894801)uu_{x}  rmse =  0.03214569775999673
Dataset is using device:  cuda
 19750  MSE: 4.12e-04  Reg: 6.34e-06  L1: 1.71e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098992)u_{xx} + (-1.110622)uu_{x} + (0.255400)u^2u_{x}  rmse =  0.0803467926175979
Dataset is using device:  cuda
  6000  MSE: 4.04e-04  Reg: 1.04e-05  L1: 1.42e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097118)u_{xx} + (-0.989942)uu_{x}  rmse =  0.003020264457517421
Dataset is using device:  cuda
 19500  MSE: 3.99e-04  Reg: 5.93e-06  L1: 1.45e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097440)u_{xx} + (-0.996900)uu_{x}  rmse =  0.0011605358132145886
Dataset is using device:  cuda
 19750  MSE: 4.06e-04  Reg: 8.14e-06  L1: 1.48e+00 Algorithm converged. Writing model to disk.
Predict

## Testing 2.4

In [9]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(17,19):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  60
Dataset is using device:  cuda
 11500  MSE: 5.55e-04  Reg: 1.41e-05  L1: 1.44e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.094623)u_{xx} + (-0.985206)uu_{x}  rmse =  0.004544083650905806
Dataset is using device:  cuda
 13000  MSE: 5.51e-04  Reg: 1.34e-05  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098568)u_{xx} + (-0.982549)uu_{x}  rmse =  0.005054712589217675
Dataset is using device:  cuda
  8000  MSE: 5.41e-04  Reg: 1.26e-05  L1: 1.48e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.099602)u_{xx} + (-0.992909)uu_{x}  rmse =  0.0020503381588241193
Dataset is using device:  cuda
  8750  MSE: 5.48e-04  Reg: 1.13e-05  L1: 1.42e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097134)u_{xx} + (-0.984439)uu_{x}  rmse =  0.004567754758948848
Dataset is using device:  cuda
 11750  MSE: 5.61e-04  Reg: 8.58e-06  L1: 1.45e+00 Algorithm converged. Writing model to disk.
Pre

KeyboardInterrupt: 

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(18,19):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  50
Dataset is using device:  cuda


2023-05-03 10:12:46.018849: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 10:12:46.658132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  8750  MSE: 6.55e-04  Reg: 1.50e-05  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.035045)u_{x} + (0.091310)u_{xx} + (-0.895245)uu_{x}  rmse =  0.03198591626743958
Dataset is using device:  cuda
 18250  MSE: 6.64e-04  Reg: 8.04e-06  L1: 1.42e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.098565)u_{xx} + (-0.990170)uu_{x}  rmse =  0.002867695802893467
Dataset is using device:  cuda
  6000  MSE: 6.55e-04  Reg: 9.27e-06  L1: 1.40e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.095379)u_{xx} + (-0.982462)uu_{x}  rmse =  0.005235518337744499
Dataset is using device:  cuda
 14500  MSE: 6.59e-04  Reg: 6.88e-06  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097574)u_{xx} + (-0.985401)uu_{x}  rmse =  0.00427216303985483
Dataset is using device:  cuda
 19000  MSE: 6.43e-04  Reg: 7.76e-06  L1: 1.44e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.097

## Testing 2.5

In [8]:
# Burgers eqn ut = -1uux +0.1uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.1
true_coeff[5]=-1

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("2_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(19,21):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))

SNR =  40
Dataset is using device:  cuda


2023-05-05 10:53:40.949933: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 10:53:41.578901: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  9750  MSE: 8.06e-04  Reg: 1.49e-05  L1: 1.44e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.096075)u_{xx} + (-0.985240)uu_{x}  rmse =  0.004408916879242191
Dataset is using device:  cuda
 19750  MSE: 8.51e-04  Reg: 1.02e-05  L1: 1.43e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.100790)u_{xx} + (-0.976331)uu_{x}  rmse =  0.006836335118891905
Dataset is using device:  cuda
  7500  MSE: 8.37e-04  Reg: 1.01e-05  L1: 1.41e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.100328)u_{xx} + (-0.987450)uu_{x}  rmse =  0.003624002365310926
Dataset is using device:  cuda
 19750  MSE: 7.69e-04  Reg: 5.58e-06  L1: 1.67e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.094555)u_{xx} + (-1.101847)uu_{x} + (0.216581)u^2u_{x}  rmse =  0.06910729437272616
Dataset is using device:  cuda
 19750  MSE: 8.02e-04  Reg: 1.29e-05  L1: 1.45e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.